In [1]:
import pandas as pd
import numpy as np
import os
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from pprint import pprint

**Dataset configuration**

In [2]:
dataset = 'robotics-final'

dataset_path = '../datasets/' + dataset + '/'
output_path = '../datasets/' + dataset + '-processed/'
features_output_path = '../datasets/' + dataset + '-features/'

**Merge the individual CSVs into one DataFrame per device for the chosen devices**

In [3]:
data_frames = {}

devices_to_use = [
    '128.237.246.127',
    '128.237.248.186',
    '128.237.247.134',
    '128.237.254.195', # this Mite is only in the Synergy kitchen dataset
    'DialogIoT 591844595',
    'DialogIoT 591844599',
    'DialogIoT 591844765',
    'Matrix b827eb96f31a',
    'Matrix b827ebe6e0f8',
    'Matrix b827eb41f96f',
    'TI SensorTag 604',
    'TI SensorTag 690',
    'TI SensorTag 85',
    'xdk_1',
    'xdk_2',
    'xdk_3'
]

columns_to_rename = {
    'ACCEL_sst_0_avg': 'accel_x',
    'ACCEL_sst_1_avg': 'accel_y',
    'ACCEL_sst_2_avg': 'accel_z',
    'MAGNETOMETER_sst_0_avg': 'mag_x',
    'MAGNETOMETER_sst_1_avg': 'mag_y',
    'MAGNETOMETER_sst_2_avg': 'mag_z',
    'HUMIDITY_sst_0_avg': 'humidity',
    'ILLUMINATION_sst_0_avg': 'light',
    'BAROMETER_sst_0_avg': 'pressure',
    'TEMPERATURE_sst_0_avg': 'temperature',
    'MICROPHONE_sst_0_avg': 'microphone',
    'microphone_avg': 'microphone',
}

columns_to_keep = [
   'accel_x',
   'accel_y',
   'accel_z',
   'gyro_x',
   'gyro_y',
   'gyro_z',
   'mag_x',
   'mag_y',
   'mag_z',
   'humidity',
   'light',
   'pressure',
   'temperature',
   'microphone'
]

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            df = pd.DataFrame()
            if device_id in data_frames:
                df = data_frames[device_id]
            
            df_new = pd.DataFrame.from_csv(dataset_path + file)
            df_new.rename(index=str, columns=columns_to_rename, inplace=True)
            cols = []
            for column in df_new:
                if column in columns_to_keep:
                    cols.append(column)

            if len(cols) > 0:
                df_new = df_new[cols]
                df = df.join(df_new, how='outer')

                data_frames[device_id] = df

**Remove duplicate indices**

In [4]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df[~df.index.duplicated(keep='first')]
    data_frames[device_id] = df

**Convert index to datetime**

In [5]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.index = pd.to_datetime(df.index)
    data_frames[device_id] = df

**Fill missing values**

In [6]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df.ffill().bfill()
    data_frames[device_id] = df

**Create 5 sec activity windows**

In [7]:
window_size_secs = 5 # seconds
smallest_size_secs = 3 # seconds

activities = pd.DataFrame.from_csv(dataset_path + 'activities.csv')
labels = pd.DataFrame.from_csv(dataset_path + 'activity_labels.csv')
labeled_activities = activities.loc[activities.id != -1]

activity_windows = []

for id in labeled_activities.id.unique():
    start = labeled_activities.loc[labeled_activities.id == id].index.min()
    end = labeled_activities.loc[labeled_activities.id == id].index.max()

    since = start
    until = start + pd.DateOffset(seconds=window_size_secs)

    while since < end:
        label = labels.loc[id]['label']

        length = (until - since).seconds
        if length >= smallest_size_secs:
            activity_windows.append({
                'since': since,
                'until': until,
                'label': label,
                'length': length,
                'id': id
            })

        since = until
        until = until + pd.DateOffset(seconds=window_size_secs)
        until = min(until, end)

**Create per-activity dataframes which are not missing more than 1 second of data**

In [8]:
activity_data_frames = {}
device_labels = {}

removed_activities = {}
all_activities = {}
activity_id = 0

for device_id in data_frames:
    df = data_frames[device_id]
    data_frames[device_id] = None
    df['id'] = -1

    device_activities = []

    for i, window in enumerate(activity_windows):

        activity_df = df.loc[df.index >= window['since']]
        activity_df = activity_df.loc[activity_df.index <= window['until']].copy()

        seconds = activity_df.index.round('s').unique()

        key = device_id + ', activity ' + str(window['label'])
        if not key in all_activities:
            all_activities[key] = 0
            removed_activities[key] = 0

        if len(seconds) + 1 < window['length']:
            removed_activities[key] += 1
        else:
            activity_df['id'] = activity_id
            activity_data_frames[activity_id] = activity_df
            device_activities.append(pd.DataFrame({
                'id': [activity_id],
                'label': [window['label']],
                'activity_id': [window['id']],
                'since': [window['since']],
                'until': [window['until']],
                'window_id': [i]
            }))
            activity_id += 1

        all_activities[key] += 1

    df_labels = pd.concat(device_activities)
    df_labels = df_labels.set_index('id')
    device_labels[device_id] = df_labels

# print stats    
for key in all_activities:
    if removed_activities[key] > 0:
        print(str(removed_activities[key]) + ' out of ' + str(all_activities[key]) + ' removed for ' + key)

1 out of 74 removed for 128.237.246.127, activity 1
27 out of 74 removed for DialogIoT 591844599, activity 8


**Resample to 10Hz and fill missing values**

In [9]:
for activity_id in activity_data_frames:
    df = activity_data_frames[activity_id]
    df = df.resample('100L')
    df = df.ffill().bfill()
    activity_data_frames[activity_id] = df

**Update device data frames**

In [10]:
for device_id in device_labels:
    labels = device_labels[device_id]
    df_parts = [activity_data_frames[id] for id in labels.index]
    df = pd.concat(df_parts)
    data_frames[device_id] = df

**Replace timestamp indices with integer ones**

In [11]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.insert(0, 'i_index', pd.Series(range(len(df.index)), index=df.index))
    df = df.set_index(['i_index'])
    data_frames[device_id] = df

**Save as Pickle files to disk**

In [12]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.to_pickle(output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(output_path + device_id + '_labels.p')

**Extract features and store them on disk**

In [13]:
data_frame_features = {}

def extract_device_features(df):
    extracted_features = extract_features(df, column_id="id")
    impute(extracted_features)
    return extracted_features

for device_id in data_frames:
    df = data_frames[device_id]

    extracted = extract_device_features(df)
    data_frame_features[device_id] = extracted

Feature Extraction: 100%|██████████| 4/4 [01:20<00:00, 26.96s/it]
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_z__max_langevin_fixed_point__m_3__r_30'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_0'
 'temperature__max_langevin_fixed_point__m_3__r_30'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_3'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_2'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_1'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_x__max_langevin_fixed_point__m_3__r_30'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_2'


Feature Extraction: 100%|██████████| 11/11 [02:44<00:00, 14.99s/it]
 'temperature__friedrich_coefficients__m_3__r_30__coeff_3'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_2'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_1'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_0'
 'microphone__max_langevin_fixed_point__m_3__r_30'
 'microphone__friedrich_coefficients__m_3__r_30__coeff_3'
 'microphone__friedrich_coefficients__m_3__r_30__coeff_2'
 'microphone__friedrich_coefficients__m_3__r_30__coeff_1'
 'microphone__friedrich_coefficients__m_3__r_30__coeff_0'
 'light__max_langevin_fixed_point__m_3__r_30'
 'light__friedrich_coefficients__m_3__r_30__coeff_3'
 'light__friedrich_coefficients__m_3__r_30__coeff_2'
 'light__friedrich_coefficients__m_3__r_30__coeff_1'
 'light__friedrich_coefficients__m_3__r_30__coeff_0'
 'mag_x__max_langevin_fixed_point__m_3__r_30'
 'mag_x__friedrich_coefficients__m_3__r_30__coeff_3'
 'mag_x__friedrich_coefficients__m_3__r_30__coeff_2'
 'm

Feature Extraction: 100%|██████████| 14/14 [03:50<00:00, 16.77s/it]
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_z__max_langevin_fixed_point__m_3__r_30'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_x__max_langevin_fixed_point__m_3__r_30'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_0'
 'microphone__max_langevin_fixed_point__m_3__r_30'
 'microphone__friedrich_coefficients__m_3__r_30__coeff_3'
 'microphone__friedrich_coefficients__m_3__r_30__coeff_2'
 'microph

Feature Extraction: 100%|██████████| 4/4 [01:21<00:00, 26.32s/it]
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_0'
 'temperature__max_langevin_fixed_point__m_3__r_30'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_3'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_2'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_1'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_y__max_langevin_fixed_point__m_3__r_30'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_z__max_langevin_fixed_point__m_3__r_30'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_2'


**Sort feature columns**

In [14]:
for device_id in data_frame_features:
    df = data_frame_features[device_id]
    df = df.reindex_axis(sorted(df.columns), axis=1)
    data_frame_features[device_id] = df

**Store extracted features on disk**

In [ ]:
for device_id in data_frame_features:
    extracted = data_frame_features[device_id]
    extracted.to_pickle(features_output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(features_output_path + device_id + '_labels.p')